## Profilowanie CPU

- dekorowanie funkcji z użyciem funkcji time.time()
- cProfile i pstats
- line_profiler
- %timeit
- dis

In [7]:
# cProfiler wyświetla informacje o czaie wywołąnia fukncji

# profiling/ex1.py
def has_item(collection, item):
    found = False
    for i in collection:
        if item == i:
            found = True
    return found


#collection = range(100000000)
#print has_item(collection, 100000000)




# profiling/ex1a.py
def has_item(collection, item):
    return item in collection


#collection = range(100000000)
#print has_item(collection, 100000000)



# profiling/ex1.sh
# python -m cProfile ex1.py

# python -m cProfile ex1a.py

In [9]:
# profiling/ex2.py
# line profiler wyświetla czasy wykonania poszczagólnych linii
# do wyprofilowania funkcji, należy ją udekorować dekoratorem @profile i uruchomić przez kernprof

@profile  # ten dekorator nie jest z nikąd importowany
def has_item(collection, item):
    found = False
    for i in collection:
        if item == i:
            found = True
    return found


#collection = range(100000000)
#print has_item(collection, 100000000)


# profiling/ex2.sh
#kernprof -l -v ex2.py

NameError: name 'profile' is not defined

In [20]:
# %timeit w IPythonie pozwala na profilowanie w sesji interaktywnego interpretera

%timeit has_item(range(10000000), 10000000)

1 loop, best of 3: 467 ms per loop


In [15]:
# dis słuzy do deasemblacji kodu pythonowego na instrukcje dla maszyny wirtualnej
import dis


def f():
    x = 1
    y = []
    y.append(x)
    print y
    return x ** 2


dis.dis(f)

  6           0 LOAD_CONST               1 (1)
              3 STORE_FAST               0 (x)

  7           6 BUILD_LIST               0
              9 STORE_FAST               1 (y)

  8          12 LOAD_FAST                1 (y)
             15 LOAD_ATTR                0 (append)
             18 LOAD_FAST                0 (x)
             21 CALL_FUNCTION            1
             24 POP_TOP             

  9          25 LOAD_FAST                1 (y)
             28 PRINT_ITEM          
             29 PRINT_NEWLINE       

 10          30 LOAD_FAST                0 (x)
             33 LOAD_CONST               2 (2)
             36 BINARY_POWER        
             37 RETURN_VALUE        


## Profilowanie Pamięci

- memory_profiler

In [19]:
# profiling/ex3.py
# moduł psutil przyspiesza działąnie mempry_profilera

def has_item(collection, item):
    found = False
    for i in collection:
        if item == i:
            found = True
    return found


@profile  # tak jak w przypadku linear_profiler dekorujemy funkcję nieimportowanym dekoratorem
def main():
    collection = range(100000000)
    print has_item(collection, 100000000)


#main()


# profiling/ex3.sh
#python -m memory_profiler ex3.py

NameError: name 'profile' is not defined

## Typy wbudowane

### Lista i Krotka
lista jest alokowana z pewnym zapasem (dodatkowe puste pola), ale jeżeli zostanie zapełńiona, to zostaje zaalokowana nowa pamięć na jeszcze większą listę a oryginalna lista zostaje skopiowana z nowe miejsce. Krotka nie alokuje dodatkowego miejsca na nowe elementy (bo nie ma po co), dlatego w przypadku, kiedy lista ma byc niezmienna oszczedniej jest uż←ć krotki zamist listy

## Słownik i zbiór

Używają tablicy haszujacych do wsadzania/wyjmowania obiektów. Obiekt musi mieć zaimplementowane metody \_\_hash\_\_ i \_\_eq\_\_ żeby moża go było wsadzić do zbioru/słownika. Dzięki temu złozónosć obliczeniowa dodawania/usuwania ze zbioru/słownika to O(1) natomiast nie ma mozliwosći umieszczenia kiklu elementóœ o tym samym haszu